# Seleção de características para aprovação de crédito

In [242]:
from csv import reader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn.feature_selection import SelectFromModel

In [182]:
# fixar a semente do gerador de números aleatórios
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [183]:
# mostrar todas as colunas do dataset
pd.set_option('display.max_columns', None)

### 1. Explorando o dataset

In [184]:
# carregando o dataset
df = pd.read_csv("datasets/application_data.csv", sep=",", encoding="latin-1")
df.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [185]:
df.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,134133.000000,3.068510e+05,246546.000000,151450.00000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.061091,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.502130,5.143927e-01,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.074490,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.117850,0.087955,0.97

In [186]:
df["NAME_CONTRACT_TYPE"].value_counts()

Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64

In [187]:
df["CODE_GENDER"].value_counts()

F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64

In [188]:
df["FLAG_OWN_CAR"].value_counts()

N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64

In [189]:
df["FLAG_OWN_REALTY"].value_counts()

Y    213312
N     94199
Name: FLAG_OWN_REALTY, dtype: int64

### 2. Dividindo conjuntos de teste e treinamento

In [264]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)
print("{} train + {} test".format(len(train_set), len(test_set)))

246008 train + 61503 test


In [266]:
train_set.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
123473,243191,0,Cash loans,F,Y,N,0,171000.0,555273.0,16366.5,463500.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.035792,-23349,365243,-3595.0,-4408,31.0,1,0,0,1,0,0,NaN,1.0,2,2,TUESDAY,9,0,0,0,0,0,0,XNA,0.524685,0.358568,0.563835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2058.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
10118,111778,0,Cash loans,M,N,Y,1,157500.0,198085.5,23638.5,171000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.010032,-10921,-117,-4281.0,-3399,NaN,1,1,1,1,1,0,Laborers,3.0,2,2,SATURDAY,7,0,0,0,0,0,0,Business Entity Type 2,0.244926,0.490305,0.595456,0.0784,0.0633,0.9742,0.6464,0.0266,0.00,0.1379,0.1667,0.2083,0.0409,0.0630,0.0594,0.0039,0.0149,0.0798,0.0657,0.9742,0.6602,0.0269,0.0000,0.1379,0.1667,0.2083,0.0418,0.0689,0.0619,0.0039,0.0158,0.0791,0.0633,0.9742,0.6511,0.0268,0.00,0.1379,0.1667,0.2083,0.0416,0.0641,0.0605,0.0039,0.0153,reg oper account,block of flats,0.0645,"Stone, brick",No,1.0,0.0,1.0,0.0,-73.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
64716,175057,1,Cash loans,M,Y,Y,0,135000.0,776304.0,25173.0,648000.0,Unaccompanied,Working,Lower secondary,Civil marriage,House / apartment,0.035792,-23213,-2157,-5680.0,-5009,8.0,1,1,0,1,0,0,Drivers,2.0,2,2,FRIDAY,13,0,0,0,0,0,0,Self-employed,NaN,0.643404,0.706205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-1959.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,

### 3. Correlação entre as features

In [192]:
def correlation_color(value):
    if value == 1:
        color = 'gold'
    elif abs(value) > 0.75:
        color = 'royalblue'
    elif value < 0:
        color = 'red'
    else:
        color = 'green'
    return 'color: %s' % color

In [193]:
pd.set_option('display.max_rows', None)
correlation = train_set.corr().style.applymap(correlation_color)
correlation

In [194]:
# corr_list = []

# for col in correlation.columns:  
#     value = train_set.corrwith(col, axis=0)
#     if abs(value) > 0.75:
#             corr_list.append(col)
            
# print(corr_list)

### 4. Separando a variável dependente: TARGET

In [195]:
x_train = train_set.drop(columns=["TARGET"])
y_train = train_set["TARGET"]
x_test = test_set.drop(columns=["TARGET"])
y_test = test_set["TARGET"]

In [196]:
y_train.head(10)

123473    0
10118     0
64716     1
234940    0
236051    0
30611     0
871       0
153082    0
188110    0
278046    0
Name: TARGET, dtype: int64

### 5. Limpando o dataset (NaN)

In [197]:
x_train.isnull().sum().sort_values(ascending = False)
x_test.isnull().sum().sort_values(ascending = False)

COMMONAREA_MEDI                 42936
COMMONAREA_AVG                  42936
COMMONAREA_MODE                 42936
NONLIVINGAPARTMENTS_MODE        42646
NONLIVINGAPARTMENTS_MEDI        42646
NONLIVINGAPARTMENTS_AVG         42646
FONDKAPREMONT_MODE              42009
LIVINGAPARTMENTS_MEDI           42003
LIVINGAPARTMENTS_MODE           42003
LIVINGAPARTMENTS_AVG            42003
FLOORSMIN_AVG                   41688
FLOORSMIN_MEDI                  41688
FLOORSMIN_MODE                  41688
YEARS_BUILD_MEDI                40805
YEARS_BUILD_MODE                40805
YEARS_BUILD_AVG                 40805
OWN_CAR_AGE                     40742
LANDAREA_AVG                    36461
LANDAREA_MEDI                   36461
LANDAREA_MODE                   36461
BASEMENTAREA_AVG                35928
BASEMENTAREA_MODE               35928
BASEMENTAREA_MEDI               35928
EXT_SOURCE_1                    34854
NONLIVINGAREA_MEDI              33739
NONLIVINGAREA_AVG               33739
NONLIVINGARE

In [198]:
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_columns', 20)

In [199]:
x_train.describe(include=['O'])

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
count,246008,246008,246008,246008,244962,246008,246008,246008,246008,168902,246008,246008,77722,122469,120829,129317
unique,2,3,2,2,7,8,5,6,6,18,7,58,4,3,7,2
top,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,TUESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
freq,222490,161855,162183,170843,198741,126929,174522,157320,218389,44201,43128,54311,58964,120320,52873,127434


In [200]:
columns_with_null_values = []
    
for column in train_set:
    if train_set[column].isnull().any():
        columns_with_null_values.append(column)

len(columns_with_null_values)

67

In [201]:
x_train[columns_with_null_values].isnull().sum().sort_values(ascending = False)

COMMONAREA_MEDI                 171929
COMMONAREA_AVG                  171929
COMMONAREA_MODE                 171929
NONLIVINGAPARTMENTS_AVG         170868
NONLIVINGAPARTMENTS_MODE        170868
NONLIVINGAPARTMENTS_MEDI        170868
FONDKAPREMONT_MODE              168286
LIVINGAPARTMENTS_AVG            168196
LIVINGAPARTMENTS_MEDI           168196
LIVINGAPARTMENTS_MODE           168196
FLOORSMIN_AVG                   166954
FLOORSMIN_MEDI                  166954
FLOORSMIN_MODE                  166954
YEARS_BUILD_AVG                 163683
YEARS_BUILD_MEDI                163683
YEARS_BUILD_MODE                163683
OWN_CAR_AGE                     162187
LANDAREA_AVG                    146129
LANDAREA_MEDI                   146129
LANDAREA_MODE                   146129
BASEMENTAREA_MODE               144015
BASEMENTAREA_AVG                144015
BASEMENTAREA_MEDI               144015
EXT_SOURCE_1                    138524
NONLIVINGAREA_AVG               135943
NONLIVINGAREA_MODE       

In [202]:
drop_columns = []
columns_to_analyze = []

for column in columns_with_null_values:
    if x_train[column].isnull().sum() > len(x_train)/2:
        drop_columns.append(column)
    else:
        columns_to_analyze.append(column)
        
print(drop_columns)

['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE']


In [203]:
len(x_train.columns)

121

In [204]:
x_train = x_train.drop(columns = drop_columns)
len(x_train.columns)

80

In [205]:
x_test = x_test.drop(columns = drop_columns)
len(x_test.columns)

80

In [206]:
x_train[columns_to_analyze].isnull().sum().sort_values(ascending = False)

FLOORSMAX_MEDI                  122532
FLOORSMAX_AVG                   122532
FLOORSMAX_MODE                  122532
YEARS_BEGINEXPLUATATION_AVG     120081
YEARS_BEGINEXPLUATATION_MODE    120081
YEARS_BEGINEXPLUATATION_MEDI    120081
TOTALAREA_MODE                  118818
EMERGENCYSTATE_MODE             116691
OCCUPATION_TYPE                  77106
EXT_SOURCE_3                     48824
AMT_REQ_CREDIT_BUREAU_QRT        33211
AMT_REQ_CREDIT_BUREAU_YEAR       33211
AMT_REQ_CREDIT_BUREAU_HOUR       33211
AMT_REQ_CREDIT_BUREAU_DAY        33211
AMT_REQ_CREDIT_BUREAU_WEEK       33211
AMT_REQ_CREDIT_BUREAU_MON        33211
NAME_TYPE_SUITE                   1046
OBS_30_CNT_SOCIAL_CIRCLE           805
DEF_30_CNT_SOCIAL_CIRCLE           805
OBS_60_CNT_SOCIAL_CIRCLE           805
DEF_60_CNT_SOCIAL_CIRCLE           805
EXT_SOURCE_2                       529
AMT_GOODS_PRICE                    224
AMT_ANNUITY                         10
DAYS_LAST_PHONE_CHANGE               1
CNT_FAM_MEMBERS          

In [207]:
x_train[columns_to_analyze].describe()

,AMT_ANNUITY,AMT_GOODS_PRICE,CNT_FAM_MEMBERS,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,245998.000000,2.457840e+05,246007.000000,2.454790e+05,197184.000000,125927.000000,123476.000000,125927.000000,123476.000000,125927.000000,123476.000000,127190.000000,245203.000000,245203.000000,245203.000000,245203.000000,246007.000000,212797.00000,212797.000000,212797.000000,212797.000000,212797.000000,212797.000000
mean,27120.236898,5.389831e+05,2.152642,5.143846e-01,0.511034,0.977713,0.226512,0.977049,0.222510,0.977718,0.226118,0.102577,1.421834,0.143155,1.404787,0.100113,-962.679249,0.00648,0.006729,0.034211,0.266644,0.266921,1.901126
std,14510.664192,3.700527e+05,0.908734,1.910533e-01,0.194838,0.059673,0.144689,0.064950,0.143716,0.060396,0.145097,0.107539,2.415520,0.447389,2.394182,0.363496,827.636821,0.08424,0.106693,0.204336,0.911958,0.834133,1.870273
min,1615.500000,4.050000e+04,1.000000,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4292.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16506.000000,2.385000e+05,2.000000,3.922228e-01,0.370650,0.976700,0.166700,0.976700,0.166700,0.976700,0.166700,0.041400,0.000000,0.000000,0.000000,0.000000,-1570.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24903.000000,4.500000e+05,2.000000,5.658916e-01,0.537070,0.982100,0.166700,0.981600,0.166700,0.981600,0.166700,0.068700,0.000000,0.000000,0.000000,0.000000,-756.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,34654.500000,6.795000e+05,3.000000,6.637440e-01,0.669057,0.986600,0.333300,0.986600,0.333300,0.986600,0.333300,0.127800,2.000000,0.000000,2.000000,0.000000,-273.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.000000
max,258025.500000,4.050000e+06,20.000000,8.549997e-01,0.896010,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,348.000000,34.000000,344.000000,24.000000,0.000000,3.00000,9.000000,8.000000,27.000000,261.000000,25.000000


In [208]:
# Listas com colunas de moda e mediana
mode_list = []
median_list = []

for column in columns_to_analyze:
    suffix = column[-4:]
    if suffix == "MODE":
        mode_list.append(column)
    elif suffix == "MEDI":
        median_list.append(column)

In [209]:
# MODA
mode_list.remove('EMERGENCYSTATE_MODE')

for column in mode_list:
    x_train[column] = x_train[column].fillna(x_train[column].mode()[0])
    x_test[column] = x_test[column].fillna(x_test[column].mode()[0])

    columns_to_analyze.remove(column)

In [210]:
# MEDIANA
for column in median_list:
    x_train[column] = x_train[column].fillna(x_train[column].median())
    x_test[column] = x_test[column].fillna(x_test[column].median())
    columns_to_analyze.remove(column)

In [211]:
# Lista com colunas médias
avg_list = []

for column in columns_to_analyze:
    suffix = column[-3:]
    prefix = column[:3]
    if suffix == "AVG" or prefix == "AMT" or prefix == "EXT" or x_train[column].isnull().sum() < 1000:
        avg_list.append(column)        

In [212]:
# MÉDIA
for column in avg_list:
    x_train[column] = x_train[column].fillna(x_train[column].mean())
    x_test[column] = x_test[column].fillna(x_test[column].mean())
    columns_to_analyze.remove(column)

In [213]:
x_train[columns_to_analyze].isnull().sum().sort_values(ascending = False)

EMERGENCYSTATE_MODE    116691
OCCUPATION_TYPE         77106
NAME_TYPE_SUITE          1046
dtype: int64

In [214]:
x_train['NAME_TYPE_SUITE'] = x_train['NAME_TYPE_SUITE'].fillna('Unaccompanied')
x_train['EMERGENCYSTATE_MODE'] = x_train['EMERGENCYSTATE_MODE'].fillna('Undefined')
x_train['OCCUPATION_TYPE'] = x_train['OCCUPATION_TYPE'].fillna('Other')

x_test['NAME_TYPE_SUITE'] = x_test['NAME_TYPE_SUITE'].fillna('Unaccompanied')
x_test['EMERGENCYSTATE_MODE'] = x_test['EMERGENCYSTATE_MODE'].fillna('Undefined')
x_test['OCCUPATION_TYPE'] = x_test['OCCUPATION_TYPE'].fillna('Other')

In [215]:
x_train.isnull().any()

SK_ID_CURR                      False
NAME_CONTRACT_TYPE              False
CODE_GENDER                     False
FLAG_OWN_CAR                    False
FLAG_OWN_REALTY                 False
CNT_CHILDREN                    False
AMT_INCOME_TOTAL                False
AMT_CREDIT                      False
AMT_ANNUITY                     False
AMT_GOODS_PRICE                 False
NAME_TYPE_SUITE                 False
NAME_INCOME_TYPE                False
NAME_EDUCATION_TYPE             False
NAME_FAMILY_STATUS              False
NAME_HOUSING_TYPE               False
REGION_POPULATION_RELATIVE      False
DAYS_BIRTH                      False
DAYS_EMPLOYED                   False
DAYS_REGISTRATION               False
DAYS_ID_PUBLISH                 False
FLAG_MOBIL                      False
FLAG_EMP_PHONE                  False
FLAG_WORK_PHONE                 False
FLAG_CONT_MOBILE                False
FLAG_PHONE                      False
FLAG_EMAIL                      False
OCCUPATION_T

### 6. Ajustando variáveis categóricas

In [216]:
categoricas = list(x_train.select_dtypes('object').columns)
categoricas

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'EMERGENCYSTATE_MODE']

In [217]:
# flags = []
# categories = []

# for column in categoricas:
#     prefix = column[:4]
#     suffix = column[-4:]
#     if prefix == 'CODE' or prefix == 'FLAG' or suffix == 'MODE':
#         flags.append(column)
#     else:
#         categories.append(column)

--------------------------------------------------------------------------------------------------------------------------------

DÚVIDAS

- jeito melhor de fazer o encoding das flags?
- usar one hot encoding?


--------------------------------------------------------------------------------------------------------------------------------


In [218]:
for column in categoricas:
    x_train["{0}_CAT".format(column)] = LabelEncoder().fit_transform(x_train[column])
    x_train = x_train.drop(columns = column, axis=1)
    
x_train.head(10)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_CAT,CODE_GENDER_CAT,FLAG_OWN_CAR_CAT,FLAG_OWN_REALTY_CAT,NAME_TYPE_SUITE_CAT,NAME_INCOME_TYPE_CAT,NAME_EDUCATION_TYPE_CAT,NAME_FAMILY_STATUS_CAT,NAME_HOUSING_TYPE_CAT,OCCUPATION_TYPE_CAT,WEEKDAY_APPR_PROCESS_START_CAT,ORGANIZATION_TYPE_CAT,EMERGENCYSTATE_MODE_CAT
123473,243191,0,171000.0,555273.0,16366.5,463500.0,0.035792,-23349,365243,-3595.0,-4408,1,0,0,1,0,0,1.0,2,2,9,0,0,0,0,0,0,0.358568,0.563835,0.977713,0.226512,0.9871,0.1667,0.9816,0.1667,0.0000,0.0,0.0,0.0,0.0,-2058.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,1,0,6,3,4,5,1,12,5,57,1
10118,111778,1,157500.0,198085.5,23638.5,171000.0,0.010032,-10921,-117,-4281.0,-3399,1,1,1,1,1,0,3.0,2,2,7,0,0,0,0,0,0,0.490305,0.595456,0.974200,0.166700,0.9742,0.1667,0.9742,0.1667,0.0645,1.0,0.0,1.0,0.0,-73.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,1,6,7,4,1,1,8,2,4,0
64716,175057,0,135000.0,776304.0,25173.0,648000.0,0.035792,-23213,-2157,-5680.0,-5009,1,1,0,1,0,0,2.0,2,2,13,0,0,0,0,0,0,0.643404,0.706205,0.977713,0.226512,0.9871,0.1667,0.9816,0.1667,0.0000,2.0,0.0,2.0,0.0,-1959.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0,0,1,1,1,6,7,3,0,1,4,0,42,1
234940,372147,1,164133.0,900000.0,36787.5,900000.0,0.030755,-10703,-2530,-2618.0,-2751,1,1,1,1,1,0,3.0,2,2,10,0,0,0,0,1,1,0.426431,0.506484,0.984100,0.333300,0.9841,0.3333,0.9841,0.3333,0.1222,0.0,0.0,0.0,0.0,-531.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0,1,1,1,6,1,4,1,1,6,5,47,0
236051,373412,0,225000.0,533668.5,21294.0,477000.0,0.025164,-15798,-3520,-8006.0,-5001,1,1,0,1,0,0,2.0,2,2,12,0,0,0,0,0,0,0.445701,0.528093,0.977713,0.226512,0.9871,0.1667,0.9816,0.1667,0.0000,6.0,1.0,6.0,0.0,-9.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0,0,1,0,1,5,1,4,1,1,8,2,16,1
30611,135529,0,405000.0,675000.0,30631.5,675000.0,0.010006,-16721,-1482,-1432.0,-265,1,1,0,1,0,0,2.0,2,1,11,0,0,0,0,0,0,0.716030,0.546023,0.977713,0.226512,0.9871,0.1667,0.9816,0.1667,0.0000,4.0,0.0,4.0,0.0,-128.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,1,0,6,7,1,1,1,0,2,26,1
871,100999,0,450000.0,2125953.0,81108.0,1984500.0,0.072508,-10121,-1354,-1393.0,-2672,1,1,0,1,1,0,1.0,1,1,16,0,0,0,0,0,0,0.683097,0.511034,0.980600,0.458300,0.9806,0.4583,0.9806,0.4583,0.1356,0.0,0.0,0.0,0.0,-4.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,6,1,1,3,5,6,0,3,0
153082,277437,0,180000.0,679500.0,24201.0,679500.0,0.022800,-17676,-2535,-472.0,-1229,1,1,0,1,1,0,1.0,2,2,11,0,0,0,0,0,0,0.181508,0.746300,0.998500,0.375000,0.9985,0.3750,0.9985,0.3750,0.1867,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,6,1,4,3,1,13,0,43,0
1

In [221]:
for column in categoricas:
    x_test["{0}_CAT".format(column)] = LabelEncoder().fit_transform(x_test[column])
    x_test = x_test.drop(columns = column, axis=1)
    
x_test.head(10)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_CAT,CODE_GENDER_CAT,FLAG_OWN_CAR_CAT,FLAG_OWN_REALTY_CAT,NAME_TYPE_SUITE_CAT,NAME_INCOME_TYPE_CAT,NAME_EDUCATION_TYPE_CAT,NAME_FAMILY_STATUS_CAT,NAME_HOUSING_TYPE_CAT,OCCUPATION_TYPE_CAT,WEEKDAY_APPR_PROCESS_START_CAT,ORGANIZATION_TYPE_CAT,EMERGENCYSTATE_MODE_CAT
245895,384575,2,207000.0,465457.5,52641.0,418500.0,0.009630,-13297,-762,-637.0,-4307,1,1,0,1,0,0,4.0,2,2,11,0,0,0,0,1,1,0.604894,0.000527,0.977823,0.225365,0.9791,0.1667,0.9816,0.1667,0.0000,0.0,0.0,0.0,0.0,-2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0,1,1,0,6,1,4,1,1,15,4,5,1
98194,214010,0,247500.0,1281712.5,48946.5,1179000.0,0.006852,-14778,-1141,-1610.0,-4546,1,1,0,1,0,1,1.0,3,3,10,0,0,0,0,0,0,0.425351,0.712155,0.997000,0.416700,0.9940,0.3333,0.9970,0.4167,0.0754,2.0,0.0,2.0,0.0,-1071.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0,0,1,1,6,1,1,3,1,10,4,5,0
36463,142232,0,202500.0,495000.0,39109.5,495000.0,0.035792,-17907,-639,-2507.0,-1461,1,1,1,1,0,0,2.0,2,2,16,0,0,0,0,0,0,0.531760,0.207964,0.977823,0.225365,0.9791,0.1667,0.9816,0.1667,0.0000,5.0,0.0,5.0,0.0,-1435.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0,0,1,0,6,7,4,1,1,15,5,42,1
249923,389171,0,247500.0,254700.0,24939.0,225000.0,0.046220,-19626,-6982,-11167.0,-3158,1,1,0,1,0,0,1.0,1,1,14,0,0,0,0,0,0,0.693521,0.614414,0.984600,0.625000,0.9846,0.6250,0.9846,0.6250,0.1285,0.0,0.0,0.0,0.0,-2000.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,1,6,4,4,5,1,6,0,5,0
158389,283617,0,112500.0,308133.0,15862.5,234000.0,0.018850,-20327,-1105,-7299.0,-494,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,0,0,0.560690,0.636376,0.971700,0.166700,0.9717,0.1667,0.9717,0.1667,0.0765,0.0,0.0,0.0,0.0,-173.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0,1,0,1,6,7,4,3,1,8,6,5,0
226343,362171,0,85500.0,152820.0,16456.5,135000.0,0.006629,-19130,365243,-8549.0,-959,1,0,0,1,1,0,2.0,2,2,6,0,0,0,0,0,0,0.519127,0.104795,0.977823,0.225365,0.9791,0.1667,0.9816,0.1667,0.0000,3.0,0.0,3.0,0.0,-1011.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0,1,0,1,6,3,4,1,1,12,4,57,1
69571,180689,1,112500.0,900000.0,24750.0,900000.0,0.015221,-15722,-345,-492.0,-4672,1,1,0,1,1,0,3.0,2,2,12,0,0,0,1,0,1,0.547963,0.510129,0.977823,0.225365,0.9791,0.1667,0.9816,0.1667,0.0000,0.0,0.0,0.0,0.0,-440.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0,0,0,0,1,1,4,1,1,12,6,5,1
181469,310328,0,141606.0,810000.0,33120.0,810000.0,0.018801,-10801,-260,-263.0,-3152,1,1,1

--------------------------------------------------------------------------------------------------------------------------------

DÚVIDAS

- quais outras variáveis podem ser relevantes para incluirmos na análise?
- usar dataset de previous_applications?


--------------------------------------------------------------------------------------------------------------------------------


In [219]:
x_train.to_csv("datasets/train_dataset.csv", index=True)

### 7. Lasso Regression

In [257]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures



def experiment(msg, pipeline, X_train, y_train, X_test, y_test):
    pipeline.fit(X_train, y_train)
    
    model = pipeline.named_steps['lin_reg']
    print('{}: \nintercept = {},\ncoefs = {}'.format(msg, model.intercept_, model.coef_))
    
    y_pred = pipeline.predict(X_test)
    RMSE = np.sqrt(mean_squared_error(y_pred, y_test))
    print('RMSE: {}'.format(RMSE))
    print()
    
alpha = 1

# Test o fit da regularização lasso.
# Test o fit da regularização lasso.
poly_reg_lasso = Pipeline([
        ("std_scaler", StandardScaler()), 
        ("lin_reg", Lasso(alpha=alpha))
    ])
experiment('Regularização Lasso', reg_lasso, x_train, y_train, x_test, y_test)
    



Regularização Lasso: 
intercept = 0.08079412051640597,
coefs = [-0.  0. -0. -0. -0. -0. -0.  0. -0.  0.  0.  0.  0.  0.  0. -0. -0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0.  0.  0.  0.  0. -0. -0.  0. -0.  0. -0. -0.  0.
  0.  0. -0.  0. -0.  0. -0.  0.]
RMSE: 0.27201597064542693



### 8. Forward Stepwise Selection

In [277]:
x_train_short = x_train[0:30000]
x_test_short = x_test[0:30000]
y_train_short = y_train[0:30000]
y_test_short = y_test[0:30000]

In [282]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y_train_short,x_train_short[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(x_test_short[list(feature_set)]) - y_test_short) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [279]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in x_train_short.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [280]:
models_fwd = pd.DataFrame(columns=["RSS", "model"])

predictors = []

for i in range(1,len(x_train_short.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names



Processed  80 models on 1 predictors in 1.2151122093200684 seconds.
Processed  79 models on 2 predictors in 1.3953793048858643 seconds.
Processed  78 models on 3 predictors in 1.510258436203003 seconds.
Processed  77 models on 4 predictors in 1.6325626373291016 seconds.
Processed  76 models on 5 predictors in 2.0422754287719727 seconds.
Processed  75 models on 6 predictors in 1.853034257888794 seconds.
Processed  74 models on 7 predictors in 2.284928560256958 seconds.
Processed  73 models on 8 predictors in 2.4488494396209717 seconds.
Processed  72 models on 9 predictors in 2.5578901767730713 seconds.
Processed  71 models on 10 predictors in 2.566617012023926 seconds.
Processed  70 models on 11 predictors in 2.7113425731658936 seconds.
Processed  69 models on 12 predictors in 2.927248001098633 seconds.
Processed  68 models on 13 predictors in 3.04858660697937 seconds.
Processed  67 models on 14 predictors in 3.26340651512146 seconds.
Processed  66 models on 15 predictors in 3.445952892

In [281]:
print(models_fwd.loc[10, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:                 TARGET   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     187.1
Date:                Mon, 23 Nov 2020   Prob (F-statistic):               0.00
Time:                        15:14:19   Log-Likelihood:                -2668.9
No. Observations:               30000   AIC:                             5358.
Df Residuals:                   29990   BIC:                             5441.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
REGION_RATING_CLIE

### 9. Backward Elimination

In [283]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [284]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(x_train_short.columns)))

tic = time.time()
predictors = x_train_short.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  80 models on 79 predictors in 46.165106534957886 seconds.
Processed  79 models on 78 predictors in 37.325110912323 seconds.
Processed  78 models on 77 predictors in 34.28974366188049 seconds.
Processed  77 models on 76 predictors in 65.30765080451965 seconds.
Processed  76 models on 75 predictors in 59.45371603965759 seconds.
Processed  75 models on 74 predictors in 59.65544843673706 seconds.
Processed  74 models on 73 predictors in 58.366652727127075 seconds.
Processed  73 models on 72 predictors in 52.738723039627075 seconds.
Processed  72 models on 71 predictors in 60.850531339645386 seconds.
Processed  71 models on 70 predictors in 53.02118277549744 seconds.
Processed  70 models on 69 predictors in 52.958906412124634 seconds.
Processed  69 models on 68 predictors in 45.840383529663086 seconds.
Processed  68 models on 67 predictors in 36.911710262298584 seconds.
Processed  67 models on 66 predictors in 32.132951974868774 seconds.
Processed  66 models on 65 predictors in 2

In [285]:
print(models_bwd.loc[10, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:                 TARGET   R-squared (uncentered):                   0.129
Model:                            OLS   Adj. R-squared (uncentered):              0.129
Method:                 Least Squares   F-statistic:                              444.7
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                        0.00
Time:                        15:37:23   Log-Likelihood:                         -2670.7
No. Observations:               30000   AIC:                                      5361.
Df Residuals:                   29990   BIC:                                      5444.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

In [286]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[10, "model"].params)

-----------------
Foward Selection:
-----------------
REGION_RATING_CLIENT_W_CITY    0.010314
EXT_SOURCE_3                  -0.220968
FLAG_MOBIL                     0.238574
EXT_SOURCE_2                  -0.167734
CODE_GENDER_CAT                0.034494
NAME_EDUCATION_TYPE_CAT        0.008950
FLAG_DOCUMENT_3                0.018900
FLAG_OWN_CAR_CAT              -0.022638
DAYS_BIRTH                     0.000002
REG_CITY_NOT_WORK_CITY         0.015034
dtype: float64


In [287]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[10, "model"].params)

-------------------
Backward Selection:
-------------------
AMT_CREDIT                 1.614350e-07
AMT_GOODS_PRICE           -1.817205e-07
DAYS_BIRTH                 1.490737e-06
DAYS_EMPLOYED              7.218241e-07
FLAG_EMP_PHONE             2.802077e-01
EXT_SOURCE_2              -1.756071e-01
EXT_SOURCE_3              -2.194009e-01
CODE_GENDER_CAT            3.309777e-02
FLAG_OWN_CAR_CAT          -2.304141e-02
NAME_EDUCATION_TYPE_CAT    9.010953e-03
dtype: float64
